In [127]:
import numpy as np
import numpy.linalg as LA

def sub_matrix(M, ids):
    dims = len(ids)
    N = np.zeros((dims, dims))
    for i in range(len(ids)):
        row = ids[i]
        for j in range(len(ids)):
            col = ids[j]
            N[i,j] = M[row,col]
    return N

def DPP(L,k):
    Y = []
    dim = L.shape[1]
    Z = list(range(dim))
    C = [[] for i in range(dim) ]
    D = [L[i,i] for i in range(dim)]
    j = np.argmax(D)
    Y.append(j)
    Z.remove(j)
    
    dj = []
    dj.append(D[j])
    print("Det:{},items:{}".format(np.prod(dj),Y))

    while len(Z)>0 and len(Y)<k:
        max_pos = -1
        max_value = None
        for i in Z:
            ei = (L[j,i] - np.dot(C[j],C[i])) / np.sqrt(D[j])
            C[i].append(ei)
            D[i] = D[i] - ei * ei
            if max_pos == -1 or max_value < D[i]:
                max_pos = i
                max_value = D[i]
        
        j = max_pos
        dj.append(D[j])
        Y.append(j)
        print("Det:{},items:{}".format(np.prod(dj),Y))     
        Z.remove(j)
    return Y

def greedy(L,k):
    dim = L.shape[1]
    Y = []
    Z = list(range(dim))
    X = []
    while len(Z)>0 and len(Y)<k:
        max_pos = -1
        max_value = None
        id2det = dict()
        for i in Z:
            candidate_list = X.copy()
            candidate_list.append(i)
            matrix = sub_matrix(L,candidate_list)
            det_value = LA.det(matrix)
            id2det[i] = det_value
            if max_pos == -1 or max_value < det_value:
                max_pos = i
                max_value = det_value
        j = max_pos

        print("list:{},max:{},det:{}".format(X,max_value,id2det))
        X.append(j)
        Y.append(j)
        Z.remove(j)
    return Y 
            
    
np.random.seed(134)
x = np.random.rand(10,10) * 2
X = np.mat(np.dot(x.T, x))
print(X)
#print(LA.det(X))
#E,V = LA.eigh(X)
#print("E:\n{}\nV:\n{}".format(E,V))
#print(sub_matrix(X,[0,3,2]))
print(DPP(X,6))
print(greedy(X,6))

[[ 18.81303201  16.52360258  10.07824282  10.39288506  13.51458745
   14.57054024  14.34696292  15.6578766    7.41406334  10.52262805]
 [ 16.52360258  16.65870168  10.31031494   9.27183847  13.65820598
   14.47099459  12.4853748   14.28239892   6.18406959  11.97814485]
 [ 10.07824282  10.31031494  12.04002251   6.77720511  11.24398495
   12.39377982   7.57619823   9.08523933   6.38329328   8.92045608]
 [ 10.39288506   9.27183847   6.77720511   9.95060595   7.78970681
   10.85914719   8.86607138   9.65875928   4.33626451   7.68772019]
 [ 13.51458745  13.65820598  11.24398495   7.78970681  16.49003452
   14.36283806   8.64052887  11.16743179   6.26857821  11.87558669]
 [ 14.57054024  14.47099459  12.39377982  10.85914719  14.36283806
   19.11865135  11.34154017  15.48986444   7.63203563  14.27531867]
 [ 14.34696292  12.4853748    7.57619823   8.86607138   8.64052887
   11.34154017  12.78025525  13.28316735   5.88186171   7.56898217]
 [ 15.6578766   14.28239892   9.08523933   9.65875928  

In [117]:
x = [[2,3],[4,5]]
a = x[0][1]
b = x[0:][0]
print(a,b)


3 [2, 3]


In [142]:
def dpp_sliding_window(L, window_size, k):
    dim = L.shape[1]
    V = np.zeros((window_size,window_size))
    C = np.zeros((dim,window_size-1))
    D = np.array([L[i,i] for i in range(dim)])
    Y = []
    Z = [i for i in range(dim)]
    
    j = np.argmax(D)
    Y.append(j)
    Z.remove(j)
    
    iter = 0
    while len(Z)>0 and len(Y)< k:
        # update V
        row = iter
        if iter >= window_size:
            V1 = np.zeros((window_size,window_size))
            V1[:-1,:-1] = V[0:,0:]
            V = V1
            row = window_size - 1
            
        c = C[j].tolist()
        if iter < window_size-1:
            c[iter] = np.sqrt(D[j])
        else:          
            c.append(np.sqrt(D[j]))
            
        #print("row:{},c:{}".format(row,c))
        for i in range(row+1):
            #print("row:{},i:{}".format(row,i))
            V[row,i] = c[i]
        
        print("V:\n",V) 
        print(D)
        print(Z)
        for i in Z:
            ei = (L[j,i] - np.dot(C[j],C[i])) / np.sqrt(D[j])
            if iter >= window_size - 1:
                C[i,-1] = ei
            else:
                C[i,row] = ei
            D[i] = D[i] - ei * ei
        #print("iter:{},D:{}".format(iter,D))
        
        if len(Y) >= window_size:
            v = V[1:,1].copy()
            V = V[1:,1:]
            A = C[:,0].copy()
            C1 = np.zeros((dim,window_size-1))
            C1[:,0:-1] = C[:,1:]
            #print("C1:{},C:{}".format(C1,C))
            C = C1
            #print(C.shape)
            for l in range(window_size - 1):
                t2 = V[l,l]**2 + v[l] ** 2
                t = np.sqrt(t2)
                V[l+1:,l] = (V[l+1:,l]*V[l,l] + v[l+1:]*v[l]) / t
                v[l+1:] = (v[l+1:] * t - V[l+1:,l] * v[l]) / V[l,l]
                for i in Z:
                    C[i,l] = (C[i,l]*V[l,l] + A[i]*v[l]) / t
                    A[i] = (A[i]*t - C[i][l]*v[l]) / V[l,l]
                V[l,l] = t
            print("1D:",D)
            print("A:\n",A)
            for i in Z:
                D[i] = D[i] + A[i]*A[i]
            print("2D:",D)
    
            
        # find the index in D
        kv = list(zip(Z,D[Z]))
        max_pos,max_value = max(kv, key=lambda x:x[1])      
        #print(max_pos, max_value, kv)
        j = max_pos
        Y.append(j)
        Z.remove(j)
        iter += 1
        print("solution:{}".format(Y))
        
dpp_sliding_window(X,4,8)

V:
 [[ 4.372488  0.        0.        0.      ]
 [ 0.        0.        0.        0.      ]
 [ 0.        0.        0.        0.      ]
 [ 0.        0.        0.        0.      ]]
[ 18.81303201  16.65870168  12.04002251   9.95060595  16.49003452
  19.11865135  12.78025525  16.40643083   6.57497468  14.78586236]
[0, 1, 2, 3, 4, 6, 7, 8, 9]
iter:0,D:[  7.70865864   5.70553972   4.00568078   3.78275051   5.69998906
  19.11865135   6.05224232   3.85659685   3.52831795   4.1269137 ]
solution:[5, 0]
V:
 [[ 4.372488    0.          0.          0.        ]
 [ 3.33232252  2.77644713  0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]]
[  7.70865864   5.70553972   4.00568078   3.78275051   5.69998906
  19.11865135   6.05224232   3.85659685   3.52831795   4.1269137 ]
[1, 2, 3, 4, 6, 7, 8, 9]
iter:1,D:[  7.70865864   1.78837847   3.95373402   3.2013633    4.84416904
  19.11865135   1.83240212   1.93088537   3.19721825   4.11040